# Auto-ablation with Howso Engine

> Note: This feature is experimental and is not universally recommended for production deployment.

## Overview

This notebook provides an overview of using auto-ablation during train to reduce the size of your data _as they are trained_.

In [1]:
import pandas as pd
from pmlb import fetch_data

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Step 1: Load Data & Feature Mapping

Our example dataset for this recipe is the well-known `Adult` dataset. This dataset works well because it has over 48,000 cases. In general, datasets containing over 25,000 rows work well. The default minimum size is 1,000 cases, below which no cases will be ablated.

This part of the process is identical to other recipes.

In [2]:
df = fetch_data("adult", local_cache_dir="../../../data")
features = infer_feature_attributes(df)

display(df)
display(features.to_dataframe())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39.0,4,215419.0,9,13.0,0,10,1,4,0,0.0,0.0,36.0,39,1
48838,64.0,0,321403.0,11,9.0,6,0,2,2,1,0.0,0.0,40.0,39,1
48839,38.0,4,374983.0,9,13.0,2,10,0,4,1,0.0,0.0,50.0,39,1
48840,44.0,4,83891.0,9,13.0,0,1,3,1,1,5455.0,0.0,40.0,39,1


type decimal_places  bounds                        \
                                              min        max allow_null   
age             continuous              0     7.0      148.0       True   
workclass          nominal              0     NaN        NaN      False   
fnlwgt          continuous              0  8103.0  3269017.0       True   
education          nominal              0     NaN        NaN      False   
education-num   continuous              0     1.0       20.0       True   
marital-status     nominal              0     NaN        NaN      False   
occupation         nominal              0     NaN        NaN      False   
relationship       nominal              0     NaN        NaN      False   
race               nominal              0     NaN        NaN      False   
sex                nominal              0     NaN        NaN      False   
capital-gain    continuous              0     0.0   162755.0       True   
capital-loss    continuous              0     0.0     8103.0       True   
hours-per-week  continuous              0     1.0      148.0       True   
native-country     nominal              0     NaN        NaN      False   
target             nominal              0     NaN        NaN      False   

               data_type original_type       
                             data_type size  
age               number       numeric    8  
workclass         number       integer    8  
fnlwgt            number       numeric    8  
education         number       integer    8  
education-num     number       numeric    8  
marital-status    number       integer    8  
occupation        number       integer    8  
relationship      number       integer    8  
race              number       integer    8  
sex               number       integer    8  
capital-gain      number       numeric    8  
capital-loss      number       numeric    8  
hours-per-week    number       numeric    8  
native-country    number       integer    8  
target            number       integer    8

## Step 3: Create Trainee and Set Parameters

The process of creating the `Trainee` is the same as the other recipes. However, after the `Trainee` is created, we set auto-analyze and auto-ablation parameters to ensure that the `Trainee` remains informed as it removes cases during train.

In [3]:
t = Trainee(features=features)

# Set auto-analyze parameters. Note that use_case_weights=True since case-weights are how
# information from ablated cases is retained in the Trainee.
t.set_auto_analyze_params(auto_analyze_enabled=True, use_case_weights=True)

# Set auto-ablation parameters.
t.set_auto_ablation_params(
    auto_ablation_enabled=True,
    influence_weight_entropy_threshold=0.2,
    delta_threshold_map={
        "accuracy": {"target": 0.1}
    }
)

Version 30.1.0 of Howso Engine™ is available. You are using version 30.0.9.dev9+g369bbb6.d20241016.
The following parameters from your configuration will override the default Amalgam parameters: {'trace'}


This recipe uses the following parameters for auto-ablation:

- `auto_ablation_enabled` — When this parameter is `False`, auto-ablation will not be performed.
- `influence_weight_entropy_threshold` — This parameter is used to determine which cases will be ablated.
  If the entropy of a case's influence weights would be above this percentile of existing cases' influence
  weight entropies, then the case is ablated. This is set to $0.2$, a very aggressive value, because of the
  next parameter, which acts as a guardrail.
- `delta_threshold_map` — This parameter defines thresholds that are used to determine when ablation should stop.
  In this case, the threshold map instructs auto-ablation to stop if the delta between the current accuracy
  of the `"target"` feature and its previous accuracy is greater than $0.1$. I.e., ablation will stop if accuracy
  for that feature drops from $0.8$ to $0.7$. More formally, ablation will stop if
  $\text{acc}_\text{old} - \text{acc}_\text{new} \geq 0.1$.

## Step 4: Training

Now that the parameters are set, the `Trainee` can be trained as if this were a normal workflow.

In [4]:
t.train(df)

Notably, only a subset of the cases have been trained (the rest have been ablated).

In [5]:
num_trained_cases = t.get_num_training_cases()
num_total_cases = len(df)
num_trained_percent = round((num_trained_cases / num_total_cases) * 100, 3)

print(f"Trained {num_trained_cases:,} cases out of {num_total_cases:,} ({num_trained_percent}%).")

Trained 13,947 cases out of 48,842 (28.555%).


## Step 5: Results

An ablated `Trainee` can be used in much the same way as other `Trainee`s, as long as `use_case_weights=True`. Here, we will investigate the prediction stats for all features.

In [6]:
stats = t.react_aggregate(
    action_feature="target",
    details={
        "prediction_stats": True,
        "selected_prediction_stats": ["all"],
    },
    use_case_weights=True,
)

In [7]:
stats.loc[stats.index != "confusion_matrix", :]

,native-country,workclass,capital-loss,fnlwgt,age,race,target,relationship,capital-gain,education-num,occupation,hours-per-week,education,sex,marital-status
precision,0.196212,0.484005,NaN,NaN,NaN,0.464559,0.776783,0.61473,NaN,NaN,0.355991,NaN,0.471767,0.783388,0.56929
mae,0.16248,0.319835,145.374775,79703.542364,7.739513,0.203318,0.161157,0.330112,970.341500,0.377296,0.743626,7.507009,0.509154,0.239506,0.238529
accuracy,0.894949,0.779559,NaN,NaN,NaN,0.867,0.867,0.741,NaN,NaN,0.32,NaN,0.593594,0.794,0.816
recall,0.109681,0.319657,NaN,NaN,NaN,0.25669,0.728052,0.611043,NaN,NaN,0.288954,NaN,0.312704,0.782854,0.423146
mcc,0.331005,0.375202,NaN,NaN,NaN,0.199941,0.502477,0.654602,NaN,NaN,0.243004,NaN,0.462654,0.566242,0.71654
spearman_coeff,NaN,NaN,0.550998,0.151643,0.665920,NaN,NaN,NaN,0.549046,0.924247,NaN,0.452625,NaN,NaN,NaN
adjusted_smape,NaN,NaN,66.963124,43.761749,20.131398,NaN,NaN,NaN,75.848423,4.473193,NaN,20.519299,NaN,NaN,NaN
rmse,NaN,NaN,402.270748,103943.330770,10.494007,NaN,NaN,NaN,4298.152245,1.056613,NaN,10.849944,NaN,NaN,NaN
smape,NaN,NaN,67.313252,43.761888,20.417853,NaN,NaN,NaN,76.273807,4.800566,NaN,20.840628,NaN,NaN,NaN
r2,NaN,NaN,-0.047190,-0.057421,0.402231,NaN,NaN,NaN,-0.365670,0.822201,NaN,0.176987,NaN,NaN,NaN


In [8]:
# The confusion matrix can be retrieved
print("Howso Prediction Results - Confusion Matrix for 'target'")
matrix = pd.DataFrame(stats["target"]["confusion_matrix"]["matrix"])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

Howso Prediction Results - Confusion Matrix for 'target'


Actual,0,1
Predicted,,
0,90,48
1,85,777


## Conclusion

Howso Engine with auto-ablation can compress data to a fraction of its original size while ensuring accuracy (and other measures) remain in an acceptable range without compromising the other capabilities of the `Trainee`.